In [1]:
import numpy as np
import netCDF4 as nc
import pandas as pd

data_dir = '/mnt/lts/nfs_fs02/sadow_lab/personal/yusukemh/PI-CASC'

In [158]:
# process skn location data
df_locations = pd.read_csv(f'{data_dir}/SKNlocations.csv')
df_locations.columns = map(str.lower, df_locations.columns)
df_locations = df_locations.rename(columns={'lat_dd': 'lat', 'lon_dd': 'lon'})

# convert the longitude from (-180, 180) to (0, 360)
df_locations['lon_updated'] = df_locations['lon'] + 360
# for each row, compute the coordinates of the closest girdpoint in netCDF files
def closest_grid(lat, lon):
    # to save computation, only include grid points around Hawaii
    lat_grids = np.arange(12.5, 27.5, 2.5)
    lon_grids = np.arange(200, 212.5, 2.5)
    
    xx, yy = np.meshgrid(lat_grids, lon_grids)
    d = np.sqrt((xx-lat)**2 + (yy-lon)**2)
    target_idx = np.where(d == d.min())
    
    return np.array([xx[target_idx], yy[target_idx]]).reshape(-1,)

df_locations['closest_grid'] = df_locations.apply(lambda row: closest_grid(row['lat'], row['lon_updated']), axis=1)

In [159]:
df_locations.head()

,skn,name,lat,lon,lon_updated,closest_grid
0,1.0,KALAE,18.916176,-155.674994,204.325006,"[20.0, 205.0]"
1,1.1,MORSE FIELD,18.913680,-155.680550,204.319450,"[20.0, 205.0]"
2,1.2,KALAE S TRK STA,18.938669,-155.680549,204.319451,"[20.0, 205.0]"
3,2.0,MANUKA,19.108660,-155.825545,204.174455,"[20.0, 205.0]"
4,2.1,KAHUKU MAUKA 2.10,19.108890,-155.746670,204.253330,"[20.0, 205.0]"


In [264]:
# process rainfall data
df_data = pd.read_excel(f"{data_dir}/FilledDataset2012.xlsx", sheet_name='Data_in')
df_data.columns = map(str.lower, df_data.columns)
# melt cells into rows
df_data = df_data.melt(
    id_vars=['skn', 'year'],
    var_name='month',
    value_name='data_in'
)

# convert string month to digit
month_to_digit = dict(
    jan=1, feb=2, mar=3, apr=4, may=5, jun=6,
    jul=7, aug=8, sep=9, oct=10, nov=11, dec=12
)
df_data.month = df_data.month.map(month_to_digit)
# replace empty space to NaN so the downstream process can remove those values
df_data['data_in'] = df_data['data_in'].replace(r'^\s*$', np.nan, regex=True)

In [265]:
df_data.head()

,skn,year,month,data_in
0,1.0,1920,1,4.76
1,1.0,1921,1,7.78
2,1.0,1922,1,4.02
3,1.0,1923,1,11.47
4,1.0,1924,1,0


In [261]:
# we need information on how each data is collected.
df_source = pd.read_excel(f"{data_dir}/FilledDataset2012.xlsx", sheet_name='Source')
df_source.columns = map(str.lower, df_source.columns)

# melt cells into rows
df_source = df_source.melt(
    id_vars=['skn', 'year'],
    var_name='month',
    value_name='filled'
)

b_filled = {
    # filled
    'Fill_1': True, 'Fill_2': True, 'Fill_3': True, 'Fill_4': True, 'Fill_5': True, 'NRFill': True,
    # nonfilled
    'State/NCDC': False, 'NCDC': False, 'State': False, 'Hydronet': False, 'RAWS': False,
    'SCAN': False, 'USGS': False, 'Hydronet/NCDC': False, 'HaleNet': False, 'HC&S': False,
    'AlanMair': False, 'USGS/State': False, 'USGS/NCDC': False, 'AlanMair/State': False,
}
df_source.filled = df_source.filled.map(b_filled)
df_source.month = df_source.month.map(month_to_digit)

In [262]:
df_source.head()

,skn,year,month,filled
0,1.0,1920,1,True
1,1.0,1921,1,True
2,1.0,1922,1,True
3,1.0,1923,1,True
4,1.0,1924,1,True


In [266]:
assert df_source.shape == df_data.shape

In [267]:
# merge the two datasets and drop rows with any NaN
df_ = pd.merge(left=df_data, right=df_source, left_on=['skn', 'year', 'month'], right_on=['skn', 'year', 'month']).dropna()

In [272]:
df_.head()

,skn,year,month,data_in,filled
0,1.0,1920,1,4.76,True
1,1.0,1921,1,7.78,True
2,1.0,1922,1,4.02,True
3,1.0,1923,1,11.47,True
4,1.0,1924,1,0,True
